<a href="https://colab.research.google.com/github/thitirat-mnc/wongnai-restuarants-rating-classification/blob/main/WangchanBERTa_wongnai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Restuarant reviews sentiment analysis** using BERT model



In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Aug 31 17:10:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

# Prepare wongnai dataset

In [ ]:
!gdown --id 1UpSJkqHhU7RMQC47Ii-6VZuByPYfhLrj

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1UpSJkqHhU7RMQC47Ii-6VZuByPYfhLrj
To: /content/w_review_data.csv
100% 60.4M/60.4M [00:00<00:00, 268MB/s]


# 1. Load and clean raw data


In [ ]:
import pandas as pd
data = pd.read_csv('w_review_data.csv', delimiter=';', names=['text','star'])

In [ ]:
data.head()

,text,star
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [ ]:
data['star'].value_counts()

4    18770
3    12171
5     6799
2     1845
1      415
Name: star, dtype: int64

In [ ]:
data['length'] = data['text'].apply(len)
data['length'].describe()

count    40000.000000
mean       540.316825
std        532.954038
min        188.000000
25%        255.000000
50%        371.000000
75%        609.000000
max      20557.000000
Name: length, dtype: float64

In [ ]:
data = data[data['length'] < 540 + (3 * 532)]

# 2. Tokenize data
add new column with tokenized text

In [ ]:
!pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 32.7 MB/s 


In [ ]:
import pythainlp
data['tokenized'] = data['text'].apply(lambda x: '|'.join(pythainlp.word_tokenize(x)))

In [ ]:
data['tokenized'].apply(lambda x: x.count('|')).describe()

count    39230.000000
mean       146.454142
std        106.410975
min          7.000000
25%         74.000000
50%        109.000000
75%        177.000000
max        821.000000
Name: tokenized, dtype: float64

In [ ]:
data.head()

,text,star,length,tokenized
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3,355,ร้านอาหาร|ใหญ่|มาก|กก|กก|กก| |\n|เลี้ยว|เข้ามา...
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4,701,อาหาร|ที่นี่|เป็น|อาหารจีน|แคะ|ที่|หากิน|ยาก|ใ...
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3,549,ปอเปี๊ยะ|สด| |ทุกวันนี้|รู้สึก|ว่า|หากิน|ยาก| ...
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5,1042,รัาน|คัพ|เค้ก|ใน|เมือง|ไทย|มี|ไม่|มาก| |หลาย|ๆ...
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5,253,อร่อย|!!!| |เดินผ่าน|Digital| |gateway|ทุกวัน|...


##Convert stars into sentiment

In [ ]:
def star_to_sentiment(star):
  if star == 1 or star ==2:
    star = 'negative'
  elif star == 3:
    star = 'neutral'
  elif star == 4 or star == 5:
    star = 'positive'
  return star

In [ ]:
data['sentiment'] = data['star'].apply(lambda x: star_to_sentiment(x))

In [ ]:
data

,text,star,length,tokenized,sentiment
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3,355,ร้านอาหาร|ใหญ่|มาก|กก|กก|กก| |\n|เลี้ยว|เข้ามา...,neutral
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4,701,อาหาร|ที่นี่|เป็น|อาหารจีน|แคะ|ที่|หากิน|ยาก|ใ...,positive
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3,549,ปอเปี๊ยะ|สด| |ทุกวันนี้|รู้สึก|ว่า|หากิน|ยาก| ...,neutral
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5,1042,รัาน|คัพ|เค้ก|ใน|เมือง|ไทย|มี|ไม่|มาก| |หลาย|ๆ...,positive
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5,253,อร่อย|!!!| |เดินผ่าน|Digital| |gateway|ทุกวัน|...,positive
...,...,...,...,...,...
39995,รู้จักร้านนี้จากวงใน ร้านอยู่ในอาคารพาณิชย์ตรง...,4,410,รู้จัก|ร้าน|นี้|จาก|วงใน| |ร้าน|อยู่|ใน|อาคาร|...,positive
39996,ร้านซูชิอาหารญี่ปุ่น อยู่ตรงสะพานลอย เกษตรประต...,4,271,ร้าน|ซูชิ|อาหาร|ญี่ปุ่น| |อยู่|ตรง|สะพานลอย| |...,positive
39997,"""Cantina Wine Bar & Italian Kitchen"" ร้านเล็กๆ...",5,1859,"""|Cantina| |Wine| |Bar| |&| |Italian| |Kitchen...",positive
39998,ร้านเค้กน่ารักๆ ตรงชั้นล่างของห้างเซ็นทรัลลาดพ...,3,231,ร้าน|เค้ก|น่ารัก|ๆ| |ตรง|ชั้นล่าง|ของ|ห้าง|เซ็...,neutral


# 3. Split data into train, dev, and test data
3.1 only 'star' and 'tokenized text' column

3.2 split data into 70:15:15


In [ ]:
from sklearn.model_selection import train_test_split
data = data[['tokenized', 'sentiment']]
train, rest = train_test_split(data, test_size=0.2)
dev, test = train_test_split(rest, test_size=0.5)

In [ ]:
train.shape

(31384, 2)

In [ ]:
dev.shape

(3923, 2)

# 4. Save 2-column file

In [ ]:
train.to_csv('wongnai-train.csv', index=False)
dev.to_csv('wongnai-dev.csv', index=False)
test.to_csv('wongnai-test.csv', index=False)

In [ ]:
!gdown --id 13wBO3baamOfaJnFBK6xyFxsC7AD0eT-F
!gdown --id 13sU9VWS_WISO_e8wU8S7zTo04qNpixrc
!gdown --id 13sg-tQrqtMJ1vmU3DhmaYmtg792eIS1Q

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=13wBO3baamOfaJnFBK6xyFxsC7AD0eT-F
To: /content/prachatai-train.csv
100% 7.86M/7.86M [00:00<00:00, 192MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=13sU9VWS_WISO_e8wU8S7zTo04qNpixrc
To: /content/prachatai-dev.csv
100% 990k/990k [00:00<00:00, 168MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...

# 5. WangchanBERTa

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 17.4 MB/s 


In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/wongnai-train.csv')
dev_df = pd.read_csv('/content/wongnai-dev.csv')
test_df = pd.read_csv('/content/wongnai-test.csv')

train_df['tokenized'] = train_df['tokenized'].apply(lambda x: x.replace('|', ''))
dev_df['tokenized'] = dev_df['tokenized'].apply(lambda x: x.replace('|', ''))
test_df['tokenized'] = test_df['tokenized'].apply(lambda x: x.replace('|', ''))


labels = list(set(train_df['sentiment'].to_list()))
label_to_idx = {label:i for i,label in enumerate(labels)}
train_df['sentiment'] = train_df['sentiment'].apply(lambda x: label_to_idx[x])
dev_df['sentiment'] = dev_df['sentiment'].apply(lambda x: label_to_idx[x])
test_df['sentiment'] = test_df['sentiment'].apply(lambda x: label_to_idx[x])

label_list = [0] * len(label_to_idx)
for label in label_to_idx:
  label_list[label_to_idx[label]] = label

In [ ]:
label_list

['positive', 'neutral', 'negative']

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoConfig,

    Trainer,
    TrainingArguments,
)

from datasets import (
    Dataset,
    DatasetDict,
    Features, Sequence, ClassLabel, Value
)

In [ ]:
model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
config = AutoConfig.from_pretrained(model_name, num_labels=len(labels))
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           config=config)



Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

In [ ]:
features= Features({
                    #"title tokens": Sequence(Value(dtype='string')),
                    "tokenized": Value(dtype='string'),

                    "sentiment": ClassLabel(names=label_list)
                })
d = DatasetDict({'train': Dataset.from_pandas(train_df, features=features),
                 'dev': Dataset.from_pandas(dev_df, features=features),
                 'test': Dataset.from_pandas(test_df, features=features)})

In [ ]:
d

DatasetDict({
    train: Dataset({
        features: ['tokenized', 'sentiment'],
        num_rows: 31384
    })
    dev: Dataset({
        features: ['tokenized', 'sentiment'],
        num_rows: 3923
    })
    test: Dataset({
        features: ['tokenized', 'sentiment'],
        num_rows: 3923
    })
})

In [ ]:
d['train'][0]

{'tokenized': 'มาลองส้มตำดูว่าเป็นไงบ้าง ก็พบว่า อร่อยนะ เนื่องด้วยมาคนเดียวเลยสั่งได้แค่ 2 เมนู ส้มตำปูอร่อยวัตถุดิบดี ปูตัวใหญ่มีเนื้ออร่อย คอหมูย่างก็พอใช้ รสชาดยังไม่โดดเด่น แต่ที่เด่นและคิดว่าจะมาลองรอบหน้าคือเมี่ยงปลาเผา ตัวโต ดูสดใหม่ ไม่แห้งและมีคนมาสั่งเรื่อยๆเลย น่าสนใจค่ะ ใครชอบเส้นมีก๋วยเตี๋ยวนัวด้วยนะคะ',
 'sentiment': 0}

In [ ]:
def tokenize(examples):
    tokenized_inputs = tokenizer(examples["tokenized"],
                                 is_split_into_words=False,
                                 truncation=True,
                                 max_length=100)
    return tokenized_inputs
tokenized_datasets = d.map(tokenize, batched=True)


  0%|          | 0/32 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][0]['input_ids'])

['<s>',
 '▁',
 'มาลอง',
 'ส้มตํา',
 'ดูว่า',
 'เป็นไงบ้าง',
 '▁',
 'ก็พบว่า',
 '▁',
 'อร่อย',
 'นะ',
 '▁',
 'เนื่องด้วย',
 'มา',
 'คนเดียว',
 'เลย',
 'สั่ง',
 'ได้แค่',
 '▁',
 '2',
 '▁',
 'เมนู',
 '▁',
 'ส้มตํา',
 'ปู',
 'อร่อย',
 'วัตถุดิบ',
 'ดี',
 '▁',
 'ปู',
 'ตัวใหญ่',
 'มี',
 'เนื้อ',
 'อร่อย',
 '▁',
 'คอ',
 'หมู',
 'ย่าง',
 'ก็พอ',
 'ใช้',
 '▁',
 'รสชาด',
 'ยังไม่',
 'โดดเด่น',
 '▁แต่',
 'ที่',
 'เด่น',
 'และ',
 'คิดว่าจะ',
 'มาลอง',
 'รอบ',
 'หน้า',
 'คือ',
 'เมี่ย',
 'ง',
 'ปลา',
 'เผา',
 '▁',
 'ตัว',
 'โต',
 '▁ดู',
 'สดใหม่',
 '▁ไม่',
 'แห้ง',
 'และ',
 'มีคนมา',
 'สั่ง',
 'เรื่อยๆ',
 'เลย',
 '▁',
 'น่าสนใจ',
 'ค่ะ',
 '▁ใคร',
 'ชอบ',
 'เส้น',
 'มี',
 'ก๋วยเตี๋ยว',
 'นัว',
 'ด้วยนะคะ',
 '</s>']

In [ ]:
model_name.split('/')[-1]

'wangchanberta-base-att-spm-uncased'

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['tokenized', 'sentiment', 'input_ids', 'attention_mask'],
        num_rows: 31384
    })
    dev: Dataset({
        features: ['tokenized', 'sentiment', 'input_ids', 'attention_mask'],
        num_rows: 3923
    })
    test: Dataset({
        features: ['tokenized', 'sentiment', 'input_ids', 'attention_mask'],
        num_rows: 3923
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer,
                                        padding=True, pad_to_multiple_of=8)
tokenized_datasets = tokenized_datasets.remove_columns(["tokenized"])
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")
#tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

In [ ]:
tokenized_datasets['dev'][20]

{'labels': 0,
 'input_ids': [5,
  10,
  1238,
  298,
  24724,
  10,
  84,
  109,
  35,
  10,
  534,
  10,
  5203,
  5919,
  28,
  10,
  425,
  424,
  30,
  2318,
  34,
  58,
  444,
  1241,
  492,
  3659,
  414,
  12509,
  483,
  832,
  73,
  10,
  2226,
  4071,
  84,
  566,
  20076,
  34,
  10,
  754,
  323,
  28,
  1116,
  10,
  16239,
  10,
  2550,
  1267,
  1140,
  1440,
  3064,
  5015,
  10,
  566,
  10,
  21926,
  1917,
  3064,
  10,
  22900,
  21926,
  1140,
  16239,
  3584,
  877,
  10,
  372,
  1741,
  16,
  5398,
  10,
  11221,
  6410,
  12,
  12414,
  129,
  1917,
  1191,
  2145,
  1116,
  10,
  1124,
  204,
  28,
  2461,
  253,
  12414,
  162,
  9,
  1204,
  2026,
  1741,
  13,
  16239,
  81,
  3166,
  10,
  803,
  21,
  6],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items()}
[len(x) for x in samples["input_ids"]]

[80, 85, 100, 86, 70, 52, 100, 100]

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 104]),
 'attention_mask': torch.Size([8, 104])}

In [ ]:
from datasets import list_metrics, load_metric
list_metrics()

['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'brier_score',
 'cer',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'exact_match',
 'f1',
 'frugalscore',
 'glue',
 'google_bleu',
 'indic_glue',
 'mae',
 'mahalanobis',
 'matthews_correlation',
 'mauve',
 'mean_iou',
 'meteor',
 'mse',
 'pearsonr',
 'perplexity',
 'poseval',
 'precision',
 'recall',
 'rl_reliability',
 'roc_auc',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'trec_eval',
 'wer',
 'wiki_split',
 'xnli',
 'xtreme_s',
 'Vertaix/vendiscore',
 'Vlasta/pr_auc',
 'abdusahmbzuai/aradiawer',
 'angelina-wang/directional_bias_amplification',
 'cakiki/ndcg',
 'codeparrot/apps_metric',
 'cpllab/syntaxgym',
 'daiyizheng/valid',
 'erntkn/dice_coefficient',
 'gorkaartola/metric_for_tp_fp_samples',
 'hack/test_metric',
 'idsedykh/codebleu',
 'idsedykh/codebleu2',
 'idsedykh/megaglue',
 'idsedykh/metric',
 'jordyvl/ece',
 'kadirnar/bary_score',
 'kagg

In [ ]:
load_metric('accuracy')

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

In [ ]:
tokenized_datasets["train"]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 31384
})

In [ ]:
tokenized_datasets["dev"]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 3923
})

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 4            |        cudaMalloc retries: 4         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   13996 MB |   14059 MB |   24142 MB |   10145 MB |\n|       from large pool |   13992 MB |   14055 MB |   24135 MB |   10143 MB |\n|       from small pool |       4 MB |       4 MB |       7 MB |       2 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |   13996 MB |   14059 MB |   24142 MB |   10145 MB |\n|       from large pool |   13992 MB |   14055 MB |

In [ ]:
args = TrainingArguments(
    output_dir='wongnai'
)

args2 = TrainingArguments(
    output_dir='prachatai-headline-2',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='steps',
    eval_steps=100,
    #save_strategy='epoch'
)
#metric.compute(predictions=preds, references=predictions.label_ids)
import numpy as np

def compute_metrics(eval_preds):
  metric = load_metric("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args2,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets['test'])
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
predicted_labels = [labels[x] for x in preds]
gold = [labels[x] for x in predictions.label_ids]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(gold, predicted_labels))

In [ ]:
dev[dev.sentiment == 'positive']

In [ ]:
test[test.sentiment == 'neutral']